# Compare cornerplots

Checking the posteriors between jim and bilby

In [6]:
import os
import numpy as np 
import json 
import matplotlib.pyplot as plt
import corner
import h5py

In [8]:
outdir = os.path.join(os.getcwd(), 'working_1')
jim_outdir = os.path.join(outdir, 'jim')
bilby_outdir = os.path.join(outdir, 'bilby')

print(os.listdir(bilby_outdir))

# bilby_results_file = os.path.join(bilby_outdir, 'bbh_0_result.hdf5')
# with h5py.File(bilby_results_file, 'r') as f:
#     print(f.keys())

FileNotFoundError: [Errno 2] No such file or directory: '/home/thibeau.wouters/projects/jim_meets_harmonic/11d_runs/working_1/bilby'

['bbh_0_copy.ini',
 'README.md',
 'working_1',
 'datafiles',
 'template_my_batch_run.ini',
 'compare_cornerplots.ipynb',
 'template_my_batch_generation.ini',
 'my_batch_run.sh',
 'outdir',
 'generation_log',
 'bbh_0.ini',
 'setup_multiple_pbilby_injections.py',
 'my_batch_generation.sh',
 'config.ini']